In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [3]:
def get_target(df, target_name):
    return df.drop(target_name, axis=1), df[target_name]

def drop_correlated_features(df, threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    df = df.drop(to_drop, axis=1)
    return df

def drop_min_rows(df, *values):
    for value in values:
        df = df.drop(df[df['DP'] == value].index, axis=0)
    return df

def balance_classes(df):
    df_balanced = pd.DataFrame()
    min_class_count = df['DP'].value_counts().min()
    for label in df['DP'].unique():
        df_label = df[df['DP'] == label]
        if len(df_label) > min_class_count:
            df_random_sample = df_label.sample(min_class_count)
            df_balanced = pd.concat([df_balanced, df_random_sample])
        else:
            df_balanced = pd.concat([df_balanced, df_label])
    return df_balanced

In [4]:
from posixpath import split
# load dataset

df = pd.read_csv("Data.csv", header = 0, delimiter = ";")
print("drop duplicates")
df = df.drop_duplicates()
constant_features = [column for column in df.columns if df[column].nunique() == 1]
print("drop constant features = ",len(constant_features))
df.drop(columns=constant_features, inplace=True)
print("split features and target")

df_features, df_target = get_target(df, 'DP')

df_corr = drop_correlated_features(df_features, 0.9)

df = pd.concat([df_corr, df_target], axis=1)
new_df = drop_min_rows(df, -60, -140)

#new_df = balance_classes(new_df)

df_check = new_df.copy()

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2

kbest = SelectKBest(k=200, score_func=mutual_info_classif)
kbest.fit(df_check.drop('DP', axis=1), df_check['DP'])
rec = kbest.get_support()
print(rec)
list_best = []
for i, r in enumerate(rec):
    if r == True:
        list_best.append(df_check.columns[i])
        print(df_check.columns[i], kbest.scores_[i])
if 'DP' in list_best:
    list_best.remove('DP')
df_best = df_check[list_best]

# check that dataframe contain column

if df_best.Disbalance.empty == None:
    df_best =  pd.concat([df_check.Disbalance, df_best], axis=1)

df_best.head()

drop duplicates
drop constant features =  1361
split features and target
[ True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False  True  True  True  True  True
  True False False  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True False  True  True  True  True  True False  True  True
 False  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

,Disbalance,ON_LINE gen 2,ON_LINE gen 3,PG gen 3,QG gen 3,PG gen 5,PG gen 11,QG gen 11,PG gen 12,QG gen 26,...,Q_BEG line 206,Q_END line 206,Q_BEG line 208,P_BEG line 220,I_BEG line 220,Q_BEG line 300,Q_BEG line 310,I_END line 312,P_BEG line 317,I_END line 317
5,143.14,0,0,0.0,0.0,0.0,68.05,6.88,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.39,3.38,0.0,0.81,3.98
15,142.77,0,0,0.0,0.0,0.0,67.66,7.02,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.38,3.34,0.0,0.49,2.44
16,142.60,0,0,0.0,0.0,0.0,68.17,7.14,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.40,3.39,0.0,0.00,0.28
18,143.20,0,0,0.0,0.0,0.0,68.17,7.07,0.0,0.0,...,-0.47,0.5,0.14,0.0,0.0,-0.38,3.38,0.0,0.00,0.28
19,142.49,0,0,0.0,0.0,0.0,67.78,7.12,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.39,3.38,0.0,0.00,0.28


In [49]:
# Tensorflow Multiclass Classification
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, Adamax, Adam, Lion, Nadam, Ftrl
from tensorflow.keras.utils import to_categorical

# data for tensorflow model
data_labels = df_check["DP"]
data_labels = pd.factorize(data_labels)[0]
data_features = df_best
train_features, test_features, train_labels, test_labels = train_test_split(data_features, data_labels, test_size = 0.2, random_state = 42)

# make model

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(data_features.shape[1],)))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(64, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer=Adamax(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model.fit(train_features, train_labels, epochs=2000, batch_size=200)

accuracy = model.evaluate(test_features, test_labels)[1]
print("Accuracy: {:.2f}%".format(accuracy * 100))



Epoch 1/2000


/home/art/prog/MLUVrepo/envml/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/art/prog/MLUVrepo/envml/lib/python3.11/site-packages/keras/src/backend/tensorflow/nn.py:599: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2805 - loss: 42.4998 
Epoch 2/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5566 - loss: 8.4511 
Epoch 3/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5921 - loss: 1.4789
Epoch 4/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6758 - loss: 0.8570
Epoch 5/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7665 - loss: 0.5924
Epoch 6/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7981 - loss: 0.5382
Epoch 7/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8012 - loss: 0.5454
Epoch 8/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8258 - loss: 0.5082
Epoch 9/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8221 - loss: 0.4922
Epoch 10/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8261 - loss: 0.4865
Epoch 11/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8360 - loss: 0.4920
Epoch 12/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accurac